# **Modelling**

### **Prepare the Data**
+ Combine all datasets to enhance predictive power and insights.

In [1]:
import logging
import pandas as pd
import os
import sys

# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Import the load_data module
try:
    from data_loader import load_data
    logger_initialized = True
except ImportError as e:
    logger_initialized = False
    print(f"Error importing 'load_data': {e}")

# Set pandas display options for better visibility
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# Configure logging
def setup_logger(name: str = 'my_logger') -> logging.Logger:
    """
    Set up a logger with INFO level and StreamHandler.
    
    Parameters:
    -----------
    name : str
        The name of the logger.
    
    Returns:
    --------
    logging.Logger
        Configured logger instance.
    """
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    
    # Prevent duplicate handlers
    if not logger.hasHandlers():
        handler = logging.StreamHandler()
        handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    
    return logger

# Initialize logger
logger = setup_logger()
logger.info("Imported necessary libraries.")

# Check and log if 'load_data' was successfully imported
if logger_initialized:
    logger.info("'load_data' module imported successfully.")
else:
    logger.warning("'load_data' module could not be imported. Check the 'scripts' directory and file availability.")

2025-01-27 16:56:20,344 - INFO - Imported necessary libraries.
2025-01-27 16:56:20,345 - INFO - 'load_data' module imported successfully.


In [3]:
logger.info("🟢 Starting the data loading process...")
df = load_data('../data/extracted_features.csv')
if not df.empty:
    logger.info(f"✅ Data loaded successfully! The dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")
else:
    logger.warning("⚠️ Data loading completed, but the dataset is empty.")

2025-01-27 16:56:20,354 - INFO - 🟢 Starting the data loading process...
2025-01-27 16:56:20,623 - INFO - ✅ Data loaded successfully! The dataset contains 95662 rows and 14 columns.


Data successfully loaded from '../data/extracted_features.csv' with 95662 rows and 14 columns.


In [4]:
df_normalized = pd.read_csv('../data/extracted_features.csv')

In [5]:
df_normalized.columns

Index(['CustomerId', 'ProductCategory', 'ChannelId', 'Amount',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'Total_Transaction_Amount', 'Average_Transaction_Amount',
       'Transaction_Count', 'Std_Transaction_Amount', 'Transaction_Hour',
       'Transaction_Day', 'Transaction_Month', 'Transaction_Year'],
      dtype='object')

In [18]:
from credit_scoring_model import CreditScoreRFM

# Initialize RFMS object with normalized DataFrame
rfm = CreditScoreRFM(df_normalized.reset_index())

In [7]:
# Calculate Recency, Frequency, and Monetary metrics using the RFM model
rfm_df = rfm.calculate_rfm()

# Display the first five rows of the RFM DataFrame
print("Calculated RFM metrics for the first five customers:")
print("=========================================================")
rfm_df[['CustomerId', 'Recency', 'Frequency', 'Monetary']].head()

Calculated RFM metrics for the first five customers:


,CustomerId,Recency,Frequency,Monetary
0,CustomerId_4406,0,119,109921.75
2,CustomerId_4683,81,2,1000.00
3,CustomerId_988,5,38,228727.20
5,CustomerId_1432,90,1,2000.00
6,CustomerId_2858,5,29,93400.00


In [ ]:
# Merge the DataFrames
# Create a mapping dictionary
risk_label_mapping = {'Good': 1, 'Bad': 0}

# Convert Risk_Label to binary values
rfm_df['Risk_Label'] = rfm_df['Risk_Label'].map(risk_label_mapping)

rfm_data = rfm_df[['CustomerId','Recency', 'Frequency', 'Monetary', 'Risk_Label']]
# Extract unique rows
df_normalized_unique = df_normalized.drop_duplicates(subset='CustomerId', keep='first')
features = pd.merge(df_normalized_unique, rfm_data, on='CustomerId', how='left')

# Set the CustomerId to index
features.set_index('CustomerId', inplace=True)

# Drop the irrelevant features
features.drop(columns=['TransactionStartTime'], inplace=True)

target = rfm_df['Risk_Label']

In [ ]:
features.head(10)

,ProductCategory,ChannelId,Amount,PricingStrategy,FraudResult,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Transaction_Amount,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year,Recency,Frequency,Monetary,Risk_Label
CustomerId,,,,,,,,,,,,,,,,,
CustomerId_4406,-0.799047,0.746738,1000.0,-0.349252,0.0,0.170118,-0.067623,-0.311831,-0.168551,-2.155530,-0.100739,0.848684,-0.994246,2176,119,109921.75,1
CustomerId_4683,-0.799047,0.746738,500.0,-0.349252,0.0,0.165122,-0.072568,-0.444993,-0.202748,-2.155530,-0.100739,0.848684,-0.994246,2256,2,1000.00,0
CustomerId_988,4.361398,0.746738,20000.0,-0.349252,0.0,0.175567,-0.008155,-0.404020,-0.009754,-1.949214,-0.100739,0.848684,-0.994246,2180,38,228727.20,1
CustomerId_1432,-0.799047,0.746738,2000.0,-0.349252,0.0,0.165168,-0.055062,-0.446132,0.000000,-1.949214,-0.100739,0.848684,-0.994246,2265,1,2000.00,0
CustomerId_2858,-0.799047,0.746738,10000.0,2.379557,0.0,0.169360,-0.040815,-0.414264,-0.140992,-1.949214,-0.100739,0.848684,-0.994246,2180,29,93400.00,1
CustomerId_598,-0.799047,0.746738,500.0,-0.349252,0.0,0.165446,-0.054887,-0.442717,-0.165175,-1.742898,-0.100739,0.848684,-0.994246,2263,4,8060.00,0
CustomerId_1053,0.491064,0.746738,600.0,-0.349252,0.0,0.165709,-0.055415,-0.439303,-0.181262,-1.742898,-0.100739,0.848684,-0.994246,2264,7,13788.00,0
CustomerId_3052,-0.799047,0.746738,890.0,-0.349252,0.0,0.165567,-0.067997,-0.433612,-0.194972,-1.742898,-0.100739,0.848684,-0.994246,2236,12,10700.00,1
CustomerId_3105,0.491064,0.746738,500.0,-0.349252,0.0,0.172394,-0.069046,-0.220780,-0.194733,-1.742898,-0.100739,0.848684,-0.994246,2175,199,159548.00,1


**Correlation analysis to select the best features**

In [ ]:
import pandas as pd

# Calculate the correlation matrix
correlation_matrix = features.corr()

# Get the correlation values for the target column
corr_with_target = correlation_matrix['Risk_Label'].abs()


# Select features that have correlation above the threshold (excluding the target column itself)
selected_features = corr_with_target[corr_with_target > 0.1].index.tolist()
selected_features.remove('Risk_Label')  # Exclude the target column itself

# Return the DataFrame with only the selected features
features = features[selected_features]

In [ ]:
corr_with_target.sort_values(ascending=False).reset_index(name='Correlation with Risk_Label')

,index,Correlation with Risk_Label
0,Risk_Label,1.000000
1,Recency,0.353630
2,Transaction_Count,0.217522
3,Frequency,0.217522
4,Transaction_Year,0.154506
5,Transaction_Month,0.140093
6,ProductCategory,0.130532
7,PricingStrategy,0.108829
8,Transaction_Day,0.093316
9,Amount,0.088088


In [ ]:
features.columns

Index(['ProductCategory', 'PricingStrategy', 'Transaction_Count',
       'Transaction_Month', 'Transaction_Year', 'Recency', 'Frequency'],
      dtype='object')

Based on the above correlation results:

Feature Selection:

+ Focus on the features with high correlations for modeling:
+ High Correlation
+ Moderate Correlation Selected features based on the threshold of 0.1 including negative correlation


In [ ]:
# Separate target and feature variables
X = features
y = target
# Print the shapes of the separated variables
print("Separated features (X) with shape:", X.shape)
print("Separated target (y) with shape:", y.shape)

Separated features (X) with shape: (3742, 7)
Separated target (y) with shape: (3742,)


### **Split the data into train and validation set**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Check for NaN values in the target variable
if y.isnull().any():
    print("NaN values found in the target variable. Handling NaN values...")

    # Option 1: Drop NaN values
    # Combine features and target for dropping NaNs
    data = pd.concat([X, y], axis=1)
    data = data.dropna()
    
    # Separate features and target again
    X = data.drop(columns=[y.name])  # Replace y.name with the actual name of your target variable if necessary
    y = data[y.name]  # Replace y.name with the actual name of your target variable if necessary

    print("NaN values dropped. New shapes:")
    print("Features (X) shape:", X.shape)
    print("Target (y) shape:", y.shape)

    # Option 2: Fill NaN values (if needed)
    # y.fillna(y.median(), inplace=True)  # Example: Fill with median
    # print("NaN values filled.")

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train and test sets created:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Train and test sets created:
X_train shape: (2993, 7)
X_test shape: (749, 7)
y_train shape: (2993,)
y_test shape: (749,)


In [ ]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit and resample the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Print the shapes of the original and resampled datasets
print("Original training data shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\nResampled training data shapes:")
print("X_resampled shape:", X_resampled.shape)
print("y_resampled shape:", y_resampled.shape)

# Now use X_resampled and y_resampled for training your model

Original training data shapes:
X_train shape: (2993, 7)
y_train shape: (2993,)

Resampled training data shapes:
X_resampled shape: (3178, 7)
y_resampled shape: (3178,)


**Create a Pipeline**

In [ ]:
# Define models
print("Defining models...")
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}
print("Models defined:", list(models.keys()))

# Define hyperparameter grids for each model
print("\nDefining hyperparameter grids...")
param_grids = {
    'Logistic Regression': {'classifier__C': [0.01, 0.1, 1, 10, 100]},
    'Decision Tree': {'classifier__max_depth': [3, 5, 7, None]},
    'Random Forest': {'classifier__n_estimators': [50, 100, 200], 'classifier__max_depth': [None, 5, 10]},
    'Gradient Boosting': {'classifier__learning_rate': [0.01, 0.1, 0.2], 'classifier__n_estimators': [50, 100, 200]}
}
print("Hyperparameter grids defined for each model.")

# Define pipelines
print("\nCreating pipelines for each model...")
pipelines = {name: Pipeline([
    ('scaler', StandardScaler()),  # Standardize the data
    ('classifier', model)  # The model to be trained
]) for name, model in models.items()}
print("Pipelines created.")

# Set up GridSearchCV for each model pipeline
print("\nSetting up GridSearchCV for each model...")
grid_searches = {
    name: GridSearchCV(estimator=pipeline, param_grid=param_grids[name], cv=5, scoring='accuracy')
    for name, pipeline in pipelines.items()
}
print("GridSearchCV set up for each model pipeline.")

Defining models...
Models defined: ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting']

Defining hyperparameter grids...
Hyperparameter grids defined for each model.

Creating pipelines for each model...
Pipelines created.

Setting up GridSearchCV for each model...
GridSearchCV set up for each model pipeline.


### **Train the Models**

In [ ]:
# Train each pipeline with the resampled dataset
print("Starting training of models with the resampled dataset...\n")
for name, grid_search in grid_searches.items():
    print(f"Training {name} model...")
    grid_search.fit(X_resampled, y_resampled)
    print(f"{name} model trained with best parameters: {grid_search.best_params_}\n")

print("All models have been trained.")

Starting training of models with the resampled dataset...

Training Logistic Regression model...
Logistic Regression model trained with best parameters: {'classifier__C': 100}

Training Decision Tree model...
Decision Tree model trained with best parameters: {'classifier__max_depth': 5}

Training Random Forest model...
Random Forest model trained with best parameters: {'classifier__max_depth': 5, 'classifier__n_estimators': 200}

Training Gradient Boosting model...
Gradient Boosting model trained with best parameters: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100}

All models have been trained.


### **Evaluate Model Performance**

In [ ]:
# Initialize a dictionary to store performance metrics
performance_metrics = {}
y_probs = {}

print("Calculating performance metrics for each model...\n")

for name, pipeline in grid_searches.items():
    print(f"Evaluating {name} model...")
    
    # Predictions on the test set
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
    y_probs[name] = y_prob  # Store probabilities for ROC curve

    # Calculate metrics
    performance_metrics[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_prob)
    }
    
    print(f"{name} model evaluation complete. Metrics calculated:\n{performance_metrics[name]}\n")

print("All models have been evaluated.")

Calculating performance metrics for each model...

Evaluating Logistic Regression model...
Logistic Regression model evaluation complete. Metrics calculated:
{'Accuracy': 0.9025367156208278, 'Precision': 0.9577464788732394, 'Recall': 0.8542713567839196, 'F1 Score': 0.9030544488711819, 'ROC AUC': np.float64(0.9567245057194806)}

Evaluating Decision Tree model...
Decision Tree model evaluation complete. Metrics calculated:
{'Accuracy': 0.9092122830440588, 'Precision': 0.9532967032967034, 'Recall': 0.871859296482412, 'F1 Score': 0.910761154855643, 'ROC AUC': np.float64(0.9456864092542483)}

Evaluating Random Forest model...
Random Forest model evaluation complete. Metrics calculated:
{'Accuracy': 0.9038718291054739, 'Precision': 0.96045197740113, 'Recall': 0.8542713567839196, 'F1 Score': 0.9042553191489362, 'ROC AUC': np.float64(0.9693911151197583)}

Evaluating Gradient Boosting model...
Gradient Boosting model evaluation complete. Metrics calculated:
{'Accuracy': 0.9198931909212283, 'Pre

### **Compare the Models**

In [ ]:
# Create a DataFrame from the performance metrics
print("Creating a DataFrame from the performance metrics...")
performance_df = pd.DataFrame(performance_metrics).T

# Display the DataFrame
print("Performance metrics DataFrame created:\n")
performance_df

Creating a DataFrame from the performance metrics...
Performance metrics DataFrame created:



,Accuracy,Precision,Recall,F1 Score,ROC AUC
Logistic Regression,0.902537,0.957746,0.854271,0.903054,0.956725
Decision Tree,0.909212,0.953297,0.871859,0.910761,0.945686
Random Forest,0.903872,0.960452,0.854271,0.904255,0.969391
Gradient Boosting,0.919893,0.956757,0.889447,0.921875,0.969631
